Анализ аргументации

[Репозиторий на GitHub](https://github.com/dialogue-evaluation/RuArg)

[Страница на CodaLab](https://codalab.lisn.upsaclay.fr/competitions/786)

## Данные

### Загрузка

In [ ]:
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/test-no_labels.tsv

In [ ]:
import pandas as pd
train = pd.read_csv('train.tsv', sep='\t')
print(train.shape)
train.head()

In [ ]:
validation = pd.read_csv('val_empty.tsv', sep='\t')
print(validation.shape)
validation.head()

In [ ]:
test = pd.read_csv('test-no_labels.tsv', sep='\t')
print(test.shape)
test.head()

Разметка содержится только в обучающей выборке.

### Анализ

Проанализируем данные обучающей выборки.

Определим минимальную, максимальную и среднюю длину текста. Отобразим распределение на графике.

In [ ]:
lens = [len(x.split()) for x in train['text']]

max_l, min_l, mean_l = max(lens), min(lens), sum(lens)/len(lens)

print(f'Минимальная длина текста: {min_l}')
print(f'Максимальная длина текста: {max_l}')
print(f'Средняя длина текста: {mean_l:.3f}')

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt

len_counts = Counter(lens)
plt.figure(figsize = (6,3))
plt.bar(len_counts.keys(), len_counts.values())

Выведем самый длинный текст.

In [ ]:
for i in range(len(train)):
    if len(train['text'][i].split()) == max_l:
        print(train['text'][i])

Проанализируем распределение по темам:
- маски
- карантин
- вакцины

In [ ]:
plt.figure()
plt.xlabel('Тема')
plt.ylabel('Количество примеров')
plt.title('Распределение типов сущностей')
plt.bar('Маски', train[train['masks_stance'] != -1].shape)
plt.bar('Карантин', train[train['quarantine_stance'] != -1].shape)
plt.bar('Вакцины', train[train['vaccines_stance'] != -1].shape)
plt.xticks(ticks=['Маски', 'Карантин', 'Вакцины'])
plt.show()

Посмотрим на распределение текстов по классам:
- «за» (2),
- «против» (0),
- другое/ нет аргумента (1),
- неактуально (-1)

In [ ]:
import numpy as np

label2 = [train[train['masks_stance'] == 2].shape[0], train[train['masks_argument'] == 2].shape[0]]
label0 = [train[train['masks_stance'] == 0].shape[0], train[train['masks_argument'] == 0].shape[0]]
label1 = [train[train['masks_stance'] == 1].shape[0], train[train['masks_argument'] == 1].shape[0]]

r = np.arange(2)
width = 0.25
plt.bar(r, label2, color = 'lightgreen',
        width = width, label='"за"')
plt.bar(r + width, label0, color = 'lightpink',
        width = width, label='"против"')
plt.bar(r + width*2, label1, color = 'lightblue',
        width = width, label='другое/\nнет аргумента')

plt.ylabel("Количество примеров")
plt.title('Распределение классов по теме "маски"')
plt.xticks(r + width,['для позиций','для доводов'])
plt.legend()
plt.show()

In [ ]:
label2 = [train[train['quarantine_stance'] == 2].shape[0], train[train['quarantine_argument'] == 2].shape[0]]
label0 = [train[train['quarantine_stance'] == 0].shape[0], train[train['quarantine_argument'] == 0].shape[0]]
label1 = [train[train['quarantine_stance'] == 1].shape[0], train[train['quarantine_argument'] == 1].shape[0]]

r = np.arange(2)
width = 0.25
plt.bar(r, label2, color = 'lightgreen',
        width = width, label='"за"')
plt.bar(r + width, label0, color = 'lightpink',
        width = width, label='"против"')
plt.bar(r + width*2, label1, color = 'lightblue',
        width = width, label='другое/\nнет аргумента')

plt.ylabel("Количество примеров")
plt.title('Распределение классов по теме "карантин"')
plt.xticks(r + width,['для позиций','для доводов'])
plt.legend()
plt.show()

In [ ]:
label2 = [train[train['vaccines_stance'] == 2].shape[0], train[train['vaccines_argument'] == 2].shape[0]]
label0 = [train[train['vaccines_stance'] == 0].shape[0], train[train['vaccines_argument'] == 0].shape[0]]
label1 = [train[train['vaccines_stance'] == 1].shape[0], train[train['vaccines_argument'] == 1].shape[0]]

r = np.arange(2)
width = 0.25
plt.bar(r, label2, color = 'lightgreen',
        width = width, label='"за"')
plt.bar(r + width, label0, color = 'lightpink',
        width = width, label='"против"')
plt.bar(r + width*2, label1, color = 'lightblue',
        width = width, label='другое/\nнет аргумента')

plt.ylabel("Количество примеров")
plt.title('Распределение классов по теме "вакцины"')
plt.xticks(r + width,['для позиций','для доводов'])
plt.legend()
plt.show()

## Модель

В этом разделе осуществим тонкую настройку модели [Sentence RuBERT](https://huggingface.co/DeepPavlov/rubert-base-cased-sentence).

### Предобработка

Для обучения нейронной сети необходимо, чтобы метки классов были положительными числами. Создадим столбец с преобразованными метками.

In [ ]:
classes = ["quarantine", "vaccines", "masks"]
label_dict = {-1: 0, 0: 1, 1: 2, 2: 3}
for c in classes:
  train[f'raw_{c}_stance'] = train[f'{c}_stance']
  train[f'{c}_stance'] = train[f'raw_{c}_stance'].map(label_dict)
  train[f'raw_{c}_argument'] = train[f'{c}_argument']
  train[f'{c}_argument'] = train[f'raw_{c}_argument'].map(label_dict)
train.head()

Преобразуем данные в формат датасетов Hugging Face.

Поскольку разметка присутствует только в обучающей выборке, из нее необходимо выделить часть для валидации.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q datasets transformers evaluate

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(train, preserve_index=False).train_test_split(test_size=0.2)
dataset

Объединим обучающую и валидационную выборку с тестовой, в которой отсутствует разметка.

In [ ]:
from datasets import DatasetDict
dataset_dict = DatasetDict({"train": dataset["train"],
                            "validation": dataset["test"],
                            "test": Dataset.from_pandas(test[["text_id", "text"]])})
dataset_dict

Загрузим токенизатор модели.

In [ ]:
from transformers import AutoTokenizer

checkpoint = 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Применим токенизацию ко всем подвыборкам датасета. Удалим лишние столбцы.

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"])

In [ ]:
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['text_id', 'text'])
tokenized_dataset

Создадим объекты класса DataLoader для деления на батчи и паддинга.

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

val_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)

### Определение функций

Будем использовать графический процессор для вычислений.

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Будем осуществлять дообучение для классификации отдельной модели по каждой теме: «маски», «карантин», «вакцины». Для этого возьмем предобученный BERT с незамороженными весами и добавим два линейных слоя: для определения позиции и для классификации доводов.

Сложность этого решения состоит в том, что класс [BertForSequenceClassification](https://huggingface.co/docs/transformers/v4.48.2/en/model_doc/bert#transformers.BertForSequenceClassification) позволяет добавить только один линейный слой. Поэтому мы создадим свой класс CustomBertForSequenceClassification.

In [ ]:
from torch import nn

class CustomBertForSequenceClassification(nn.Module):

  def __init__(self, n_labels):
    super().__init__()
    self.bert = AutoModel.from_pretrained(checkpoint)
    self.drop = nn.Dropout(p=0.3)
    self.stance_out = nn.Linear(self.bert.config.hidden_size, n_labels)
    self.argument_out = nn.Linear(self.bert.config.hidden_size, n_labels)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    stance_logits = self.stance_out(output)
    argument_logits = self.argument_out(output)

    return {"stance": stance_logits, "argument": argument_logits}

Реализуем функцию для одной эпохи обучения.

In [ ]:
import numpy as np

def train_epoch(current_class, model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train() # переводим модель в состояние обучения

  losses = [] # значения функции потерь
  # значения accuracy
  stance_correct_predictions = 0
  argument_correct_predictions = 0

  for d in data_loader: # итерация по батчам
    input_ids = d["input_ids"].to(device) # индексы токенов
    attention_mask = d["attention_mask"].to(device) # маски внимания
    # метки классов
    stance_targets = d[f"{current_class}_stance"].to(device)
    argument_targets = d[f"{current_class}_argument"].to(device)

    # применяем модель
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # позиция максимального значения
    stance_preds = torch.argmax(outputs["stance"], dim=1)
    argument_preds = torch.argmax(outputs["argument"], dim=1)
    # подсчет функции потерь
    stance_loss = loss_fn(outputs["stance"], stance_targets)
    argument_loss = loss_fn(outputs["argument"], argument_targets)
    loss = stance_loss + argument_loss

    # количество совпадений
    stance_correct_predictions += torch.sum(stance_preds == stance_targets)
    argument_correct_predictions += torch.sum(argument_preds == argument_targets)
    losses.append(loss.item())

    loss.backward() # подсчет градиента
    optimizer.step() # обновление весов
    scheduler.step() # изменение скорости обучения
    optimizer.zero_grad() # обнуление градиентов

  return stance_correct_predictions / n_examples, argument_correct_predictions / n_examples, np.mean(losses) # accuracy, среднее значение ошибки

Также реализуем функцию для валидации.

In [ ]:
def eval_model(current_class, model, data_loader, loss_fn, device, n_examples):
  model = model.eval() # переводим модель в состояние валидации

  losses = [] # значения функцим потерь
  # значения accuracy
  stance_correct_predictions = 0
  argument_correct_predictions = 0

  with torch.no_grad(): # грандиент не считается
    for d in data_loader: # итерация по батчам
      input_ids = d["input_ids"].to(device) # индексы токенов
      attention_mask = d["attention_mask"].to(device) # маски внимания
      # метки классов
      stance_targets = d[f"{current_class}_stance"].to(device)
      argument_targets = d[f"{current_class}_argument"].to(device)

      # применяем модель
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      # позиция максимального значения
      stance_preds = torch.argmax(outputs["stance"], dim=1)
      argument_preds = torch.argmax(outputs["argument"], dim=1)
      # подсчет функции потерь
      stance_loss = loss_fn(outputs["stance"], stance_targets)
      argument_loss = loss_fn(outputs["argument"], argument_targets)
      loss = stance_loss + argument_loss

      # количество совпадений
      stance_correct_predictions += torch.sum(stance_preds == stance_targets)
      argument_correct_predictions += torch.sum(argument_preds == argument_targets)
      losses.append(loss.item())

  return stance_correct_predictions / n_examples, argument_correct_predictions / n_examples, np.mean(losses) # accuracy, среднее значение ошибки

### Обучение по классам

Напишем функцию для обучения модели, чтобы потом применить ее для каждой тематики.

Вначале загрузим предобученную модель и добавим два линейных слоя.

Затем установим количество эпох и скорость обучения. Будем использовать планировщик (`scheduler`), он регулирует скорость обучения: первые несколько шагов (`num_warmup_steps`) она может увеличиваться, а потом уменьшается. Также определим функцию потерь.

Наконец, реализуем процедуру обучения и валидации.


In [ ]:
from torch.optim import AdamW
from transformers import AutoModel
from transformers import get_linear_schedule_with_warmup

def fine_tuning(current_class):

  print(f"Trainig {current_class} model:\n")

  # Загрузка предобученной модели
  bert_model = AutoModel.from_pretrained(checkpoint)
  # Добавление линейных слоев
  model = CustomBertForSequenceClassification(n_labels = 4).to(device)

  EPOCHS = 2
  # Обучение всех слоев
  optimizer = AdamW(model.parameters(), lr=2e-5)
  total_steps = len(train_dataloader) * EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

  loss_fn = nn.CrossEntropyLoss().to(device)

  for epoch in range(EPOCHS): # итерация по эпохам
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    # обучение
    train_stance_acc, train_argument_acc, train_loss = train_epoch(current_class, model, train_dataloader, loss_fn, optimizer, device, scheduler, len(dataset["train"]))

    print(f'Train loss {train_loss} stance accuracy {train_stance_acc} argument accuracy {train_argument_acc}')

    # валидация
    val_stance_acc, val_argument_acc, val_loss = eval_model(current_class, model, val_dataloader, loss_fn, device, len(dataset["test"]))

    print(f'Val loss {val_loss} stance accuracy {val_stance_acc} argument accuracy {val_argument_acc}')
    print()

  return bert_model, model

Обучим модель для каждой из тематик: карантин, вакцины и маски.

In [ ]:
classes

In [ ]:
bert_model, quarantine_model = fine_tuning(classes[0])

In [ ]:
bert_model

In [ ]:
quarantine_model

In [ ]:
bert_model, vaccines_model = fine_tuning(classes[1])

In [ ]:
bert_model, masks_model = fine_tuning(classes[2])

### Получение предсказаний

Напишем функции для получения предсказаний обученной модели и подсчета макро F1-меры.

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  # предсказанные метки
  stance_predictions = []
  argument_predictions = []

  with torch.no_grad(): # грандиент не считается
    for d in data_loader: # итерация по батчам
      input_ids = d["input_ids"].to(device) # индексы токенов
      attention_mask = d["attention_mask"].to(device) # маски внимания

      # применяем модель
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      # позиция максимального значения
      stance_preds = torch.argmax(outputs["stance"], dim=1)
      argument_preds = torch.argmax(outputs["argument"], dim=1)

      stance_predictions.extend(stance_preds)
      argument_predictions.extend(argument_preds)

  stance_predictions = torch.stack(stance_predictions).cpu()
  argument_predictions = torch.stack(argument_predictions).cpu()

  # преобразуем обратно к исходной разметке -1, 0, 1 , 2
  reverse_label_dict = {v:k for k, v in label_dict.items()}
  stance_predictions = [reverse_label_dict[x.item()] for x in stance_predictions]
  argument_predictions = [reverse_label_dict[x.item()] for x in argument_predictions]

  return stance_predictions, argument_predictions

In [ ]:
import evaluate

def compute_metrics(preds, labels):
    metric = evaluate.load("f1")
    return metric.compute(predictions=preds, references=labels, average="macro")

Посчитаем метрику для каждой из моделей. При этом тексты с меткой -1 (неактуально) не будут учитываться при подсчете.

In [ ]:
def validation_score(current_class, model):
  val_stance_predictions, val_argument_predictions = get_predictions(model, val_dataloader)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"{current_class}_stance_predictions", val_stance_predictions)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"{current_class}_argument_predictions", val_argument_predictions)
  filtered_validation = tokenized_dataset["validation"].filter(lambda example: example[f"raw_{current_class}_stance"]!=-1)
  stance_f1 = compute_metrics(filtered_validation[f"{current_class}_stance_predictions"], filtered_validation[f"raw_{current_class}_stance"])
  argument_f1 = compute_metrics(filtered_validation[f"{current_class}_argument_predictions"], filtered_validation[f"raw_{current_class}_argument"])
  return stance_f1['f1'], argument_f1['f1']

In [ ]:
classes

In [ ]:
quarantine_stance_f1, quarantine_argument_f1 = validation_score(classes[0], quarantine_model)
print(f"Quarantine Stance F1 = {quarantine_stance_f1}")
print(f"Quarantine Argument F1 = {quarantine_argument_f1}")

In [ ]:
vaccines_stance_f1, vaccines_argument_f1 = validation_score(classes[1], vaccines_model)
print(f"Vaccines Stance F1 = {vaccines_stance_f1}")
print(f"Vaccines Argument F1 = {vaccines_argument_f1}")

In [ ]:
masks_stance_f1, masks_argument_f1 = validation_score(classes[2], masks_model)
print(f"Masks Stance F1 = {masks_stance_f1}")
print(f"Masks Argument F1 = {masks_argument_f1}")

Чтобы посчитать итоговую метрику, усредним по трем тематикам.

In [ ]:
final_stance_metrics = (quarantine_stance_f1 + vaccines_stance_f1 + masks_stance_f1) / 3
final_argument_metrics = (quarantine_argument_f1 + vaccines_argument_f1 + masks_argument_f1) / 3
print(f"Final Stance F1 = {final_stance_metrics}")
print(f"Final Argument F1 = {final_argument_metrics}")

Запишем предсказания тестовой выборки в виде датафрейма. Преобразуем их в формат .tsv, а затем заархивируем.

In [ ]:
quarantine_test_stance_predictions, quarantine_test_argument_predictions = get_predictions(quarantine_model, test_dataloader)
vaccines_test_stance_predictions, vaccines_test_argument_predictions = get_predictions(vaccines_model, test_dataloader)
masks_test_stance_predictions, masks_test_argument_predictions = get_predictions(masks_model, test_dataloader)
test_predictions = pd.DataFrame.from_dict({"masks_stance": masks_test_stance_predictions, "masks_argument": masks_test_argument_predictions,
                                           "quarantine_stance": quarantine_test_stance_predictions, "quarantine_argument": quarantine_test_argument_predictions,
                                           "vaccines_stance": vaccines_test_stance_predictions, "vaccines_argument": vaccines_test_argument_predictions})
test_predictions.head()

In [ ]:
test_predictions.to_csv("test_predictions.tsv", sep='\t', index=None)

In [ ]:
!zip test_predictions.zip test_predictions.tsv

Файл test_predictions.zip может быть загружен на платформу CodaLab для подсчета метрики на тестовой подвыборке.